# Import Dependences

In [ ]:
import os
# Find the latest version of spark 3.2.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Install the PostgreSQL driver in our Colab environment
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
# Establish a Spark session and add the Postgres driver to the filepath
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AMZreviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract Data

In [ ]:
# Read the first TSV.GZ file from an S3 bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
books_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
books_df.show()

In [ ]:
# Count the amount of rows in the dataset
print(f"Rows in dataset: {books_df.count()}")

# Transform Data

In [ ]:
# Count the amount of distinct rows in the dataset to find duplicates
print(f"Distinct rows in dataset: {books_df.distinct().count()}")

In [ ]:
# Drop unnecessary columns
books_df = books_df.drop("marketplace", "product_category", "verified_purchase", "review_headline", "review_body")

books_df.show()

In [ ]:
# Find unique customer_id counts
print(f"Unique customer_id counts: {books_df.select('customer_id').distinct().count()}")

In [ ]:
# Find instances of each customer_id
customer_id_count_df = books_df.groupBy('customer_id').count()

books_df.show()

In [ ]:
# Join count to original dataframe
books_df = books_df.join(customer_id_count_df, books_df.customer_id == customer_id_count_df.customer_id, "left").drop(books_df.customer_id)
books_df.show()

In [ ]:
# Ensure rows are preserved
print(books_df.count())

In [ ]:
# Rename count column
books_df = books_df.withColumnRenamed("count", "customer_count")
books_df.show()

In [ ]:
# Check data types
books_df.printSchema()

In [ ]:
from pyspark.sql.functions import to_date
# Cast customer_count to integer and review_date to date

books_df = books_df.withColumn("customer_count", books_df.customer_count.cast("integer"))
books_df = books_df.withColumn("review_date", books_df.review_date.cast("date"))

books_df.printSchema()

In [ ]:
# Create review_id dataframe for review_id_table in our database
review_id_df = books_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date']).dropDuplicates()
review_id_df.show() 

In [ ]:
# Create products dataframe for products in our database
products_df = books_df.select(['product_id', 'product_title']).dropDuplicates(['product_id'])
products_df.show()

In [ ]:
# Create customers dataframe for customers in our database
customers_df = books_df.select(['customer_id', 'customer_count']).dropDuplicates()
customers_df.show()

In [ ]:
# Create vine dataframe for vine_table in our database
vine_df = books_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine']).dropDuplicates()
vine_df.show()

# Load Data

In [ ]:
# Fill out aws fields
my_aws_endpoint = ''
my_aws_port_number = '5432'
my_aws_database_name = 'amazon_big_data_db'
my_aws_username = 'postgres'
my_aws_password = ''

In [ ]:
# Define the connection string
jdbc_url=f'jdbc:postgresql://{my_aws_endpoint}:{my_aws_port_number}/{my_aws_database_name}'

# Set up the configuration parameters
config = {"user": f'{my_aws_username}', 
          "password": f'{my_aws_password}', 
          "driver":"org.postgresql.Driver"}

# Choose to append the data
mode = 'append'

In [ ]:
# Write the dataframe to the appropriate table in our PostgreSQL RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write the dataframe to the appropriate table in our PostgreSQL RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write the dataframe to the appropriate table in our PostgreSQL RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write the dataframe to the appropriate table in our PostgreSQL RDS

vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)